## About

This notebook demonstrates a proof of concept Garden that can host models from Park et al.'s "End-to-end AI framework for interpretable prediction of molecular and crystal properties". ([paper](https://iopscience.iop.org/article/10.1088/2632-2153/acd434), [notebook](https://colab.research.google.com/drive/1viGlpUIGmRW-kzqERj4fGzX9udEXVrXw?usp=sharing))

Their framework allows for prediction of several different properties using different ML models. For this demo we will just look at using the Physnet model to predict zero point vibrational energy (ZPVE).

In [ ]:
!pip install garden-ai

In [ ]:
import garden_ai
garden_client = garden_ai.GardenClient()

## Data

 Skipping some data loading steps, I will use one hardcoded batch for inference from the QM9 dataset.

In [3]:
import torch

In [10]:
# @title
z = torch.Tensor([8, 6, 6, 6, 6, 6, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1, 8, 6, 6, 6, 6, 6, 8,
        6, 7, 1, 1, 1, 1, 1, 8, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 8, 6, 6, 6, 6, 8, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 8, 6, 7,
        6, 6, 6, 7, 6, 7, 1, 1, 1, 6, 7, 6, 7, 6, 6, 8, 6, 7, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 6, 6, 6, 6, 7, 8, 6, 6, 7, 1, 1, 1, 1, 1, 1, 8, 6, 8, 6, 6, 7,
        6, 6, 8, 1, 1, 1, 1, 1, 6, 6, 6, 8, 6, 6, 7, 6, 7, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 6, 6, 6, 6, 8, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6,
        6, 6, 6, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 8, 6, 6, 8, 6,
        6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 7, 6, 6, 8, 6, 7, 1, 1,
        1, 1, 6, 6, 6, 7, 6, 8, 6, 7, 8, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 8,
        6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
pos = torch.Tensor([[-1.2610e-01,  1.3920e+00, -2.7400e-02],
        [ 3.4000e-03,  1.6000e-02,  5.8000e-03],
        [ 5.8570e-01, -8.9530e-01,  1.1274e+00],
        [-7.9500e-01, -1.5765e+00,  1.2781e+00],
        [-1.3156e+00, -8.0750e-01, -3.7000e-03],
        [-1.2520e+00, -1.5919e+00, -1.2883e+00],
        [-2.8100e-02, -1.5794e+00, -1.8261e+00],
        [ 9.1620e-01, -7.8800e-01, -9.6170e-01],
        [ 1.4466e+00, -1.5560e+00,  1.7640e-01],
        [ 7.5160e-01,  1.7867e+00,  2.8000e-02],
        [ 1.1216e+00, -5.6880e-01,  2.0218e+00],
        [-7.9660e-01, -2.6663e+00,  1.2272e+00],
        [-1.3101e+00, -1.2468e+00,  2.1849e+00],
        [-2.2373e+00, -2.4430e-01,  1.4960e-01],
        [-2.1018e+00, -2.1394e+00, -1.6823e+00],
        [ 2.7520e-01, -2.1010e+00, -2.7259e+00],
        [ 1.7298e+00, -2.7250e-01, -1.4833e+00],
        [ 5.1800e-02, -8.1200e-02, -2.2000e-03],
        [-9.4000e-03,  1.1339e+00,  9.9000e-03],
        [ 1.1614e+00,  1.9857e+00,  3.7000e-03],
        [ 1.3259e+00,  3.3597e+00,  1.2600e-02],
        [ 2.7198e+00,  3.6212e+00, -4.0000e-04],
        [ 3.3870e+00,  4.9207e+00,  2.7000e-03],
        [ 4.5888e+00,  5.0768e+00, -8.6000e-03],
        [ 3.3656e+00,  2.3817e+00, -1.7400e-02],
        [ 2.4248e+00,  1.4212e+00, -1.5000e-02],
        [-9.7880e-01,  1.6720e+00,  2.3400e-02],
        [ 5.3150e-01,  4.0915e+00,  2.7200e-02],
        [ 2.6950e+00,  5.7920e+00,  1.7900e-02],
        [ 4.4213e+00,  2.1626e+00, -3.0700e-02],
        [ 2.5612e+00,  4.2140e-01, -2.4600e-02],
        [ 3.4580e-01,  4.5750e-01, -1.0397e+00],
        [-1.0040e-01,  4.4530e-01,  3.1250e-01],
        [-1.4650e+00, -2.2750e-01,  3.2940e-01],
        [-1.4546e+00, -1.7224e+00,  1.6000e-02],
        [-1.3713e+00, -2.4621e+00,  1.3409e+00],
        [-1.3355e+00, -1.4124e+00,  2.4469e+00],
        [-2.1720e+00, -3.3820e-01,  1.7076e+00],
        [-3.4341e+00, -1.1515e+00,  1.3261e+00],
        [-2.7082e+00, -2.3056e+00,  6.3780e-01],
        [ 1.2328e+00,  8.2790e-01, -1.0592e+00],
        [ 6.1080e-01, -9.8300e-02,  9.5110e-01],
        [-1.8660e-01,  1.4711e+00,  7.0810e-01],
        [-2.1010e+00,  3.1690e-01, -3.8020e-01],
        [-1.0198e+00, -2.1147e+00, -8.9260e-01],
        [-8.8220e-01, -3.4234e+00,  1.4270e+00],
        [-1.8258e+00, -1.7493e+00,  3.3670e+00],
        [-3.2520e-01, -1.0812e+00,  2.7041e+00],
        [-2.3435e+00,  5.9730e-01,  2.2405e+00],
        [-4.0099e+00, -1.4681e+00,  2.2026e+00],
        [-4.1003e+00, -6.0150e-01,  6.5320e-01],
        [-3.2276e+00, -3.1483e+00,  2.0070e-01],
        [ 4.1570e-01,  9.4370e-01,  8.2080e-01],
        [-6.2600e-02, -2.2240e-01,  2.0190e-01],
        [-1.5047e+00, -2.1840e-01, -2.8080e-01],
        [-1.8254e+00,  5.4540e-01, -1.5581e+00],
        [-1.4958e+00, -9.2020e-01, -1.6012e+00],
        [-1.8380e-01, -1.3099e+00, -1.9514e+00],
        [ 7.4140e-01, -6.5050e-01, -1.0886e+00],
        [ 1.3811e+00,  4.9830e-01, -1.7389e+00],
        [ 1.9120e+00,  1.4474e+00, -2.2538e+00],
        [ 5.1060e-01,  1.6229e+00,  1.4120e-01],
        [ 5.0000e-02, -1.0110e+00,  9.5210e-01],
        [-2.2839e+00, -3.7530e-01,  4.5490e-01],
        [-2.8632e+00,  8.2440e-01, -1.7052e+00],
        [-1.1076e+00,  1.2520e+00, -1.9583e+00],
        [-2.2243e+00, -1.6594e+00, -1.9108e+00],
        [ 1.5187e+00, -1.3787e+00, -8.3450e-01],
        [ 2.3926e+00,  2.2682e+00, -2.7275e+00],
        [ 1.8680e-01,  1.7530e-01, -6.6440e-01],
        [ 1.0490e-01,  1.2825e+00, -2.1340e-01],
        [ 1.1283e+00,  1.8751e+00,  5.5180e-01],
        [ 2.5163e+00,  1.7900e+00,  2.1800e-01],
        [ 1.7843e+00,  3.1325e+00,  1.9730e-01],
        [ 1.5157e+00,  3.7252e+00, -1.0982e+00],
        [ 1.2855e+00,  4.1883e+00, -2.1313e+00],
        [ 1.9883e+00,  4.0800e+00,  1.2757e+00],
        [ 2.1690e+00,  4.8266e+00,  2.1377e+00],
        [-7.8660e-01,  1.9282e+00, -2.7820e-01],
        [ 3.2035e+00,  1.6080e+00,  1.0381e+00],
        [ 2.7828e+00,  1.3676e+00, -7.4720e-01],
        [-2.4900e-02,  1.4338e+00, -2.4400e-02],
        [ 4.6000e-03, -8.4000e-03,  1.2800e-02],
        [ 1.0800e-02, -8.7550e-01, -1.0712e+00],
        [ 5.3300e-02, -2.1225e+00, -8.0390e-01],
        [ 1.0060e-01, -2.2147e+00,  6.6830e-01],
        [ 1.3557e+00, -2.9916e+00,  1.0939e+00],
        [ 1.3515e+00, -4.2897e+00,  5.4740e-01],
        [ 7.2100e-02, -7.8010e-01,  1.1788e+00],
        [ 1.1690e-01, -4.1270e-01,  2.3944e+00],
        [ 8.7870e-01,  1.8606e+00,  4.2710e-01],
        [-8.2800e-02,  1.7664e+00, -1.0626e+00],
        [-8.9740e-01,  1.8235e+00,  5.1290e-01],
        [-1.9200e-02, -4.6090e-01, -2.0749e+00],
        [-7.7640e-01, -2.7684e+00,  1.0221e+00],
        [ 2.2477e+00, -2.4200e+00,  7.9000e-01],
        [ 1.3714e+00, -3.0836e+00,  2.1833e+00],
        [ 1.1673e+00, -4.1820e+00, -3.9430e-01],
        [ 9.6600e-02,  6.0320e-01,  2.4938e+00],
        [ 2.1160e-01,  1.3677e+00,  7.2000e-03],
        [ 8.9200e-02, -7.9500e-02,  5.0000e-04],
        [-1.9900e-02, -1.2785e+00, -4.9000e-03],
        [-1.4640e-01, -2.7026e+00, -1.1600e-02],
        [-1.2780e+00, -3.2770e+00, -3.7000e-03],
        [ 1.1037e+00, -3.2775e+00, -2.6500e-02],
        [ 1.1502e+00, -4.6931e+00, -3.4500e-02],
        [ 2.5540e+00, -5.1050e+00, -5.0200e-02],
        [ 3.6481e+00, -5.4718e+00, -6.2600e-02],
        [ 1.2620e+00,  1.6759e+00, -9.0000e-04],
        [-2.7710e-01,  1.8042e+00, -8.7050e-01],
        [-2.6060e-01,  1.7947e+00,  8.9840e-01],
        [-1.2261e+00, -4.2936e+00, -9.5000e-03],
        [ 6.6390e-01, -5.1125e+00,  8.5700e-01],
        [ 6.4690e-01, -5.1030e+00, -9.2100e-01],
        [ 2.0700e-02,  5.4200e-02,  1.8700e-02],
        [-2.2200e-02,  1.2386e+00, -5.1000e-03],
        [-1.1457e+00,  2.0088e+00, -1.3940e-01],
        [-7.0570e-01,  3.3295e+00, -2.1210e-01],
        [ 6.0800e-01,  3.3374e+00, -4.2000e-02],
        [ 1.1751e+00,  4.5564e+00,  2.9370e-01],
        [ 4.2100e-02,  5.5072e+00,  4.6000e-03],
        [-1.2883e+00,  4.6708e+00,  8.0700e-02],
        [ 1.0993e+00,  2.0713e+00,  7.4900e-02],
        [ 2.0256e+00,  4.7992e+00, -2.0370e-01],
        [ 5.9900e-02,  6.3306e+00,  7.1980e-01],
        [ 1.4680e-01,  5.9160e+00, -1.0066e+00],
        [-2.0317e+00,  5.0454e+00, -6.2860e-01],
        [-1.7321e+00,  4.7149e+00,  1.0848e+00],
        [-1.1900e-01,  1.5798e+00,  7.0200e-02],
        [-2.1100e-02,  4.7700e-02,  1.0270e-01],
        [-1.3665e+00, -5.9850e-01, -8.5100e-02],
        [-2.3223e+00, -2.9190e-01,  9.6920e-01],
        [-2.6704e+00, -1.6817e+00,  1.1435e+00],
        [-1.7531e+00, -2.1139e+00, -1.4000e-02],
        [-2.3103e+00, -1.9751e+00, -1.3606e+00],
        [-2.0797e+00, -5.9520e-01, -1.4475e+00],
        [-2.3414e+00,  3.2630e-01, -2.2711e+00],
        [ 8.6650e-01,  2.0310e+00,  2.1650e-01],
        [-5.2380e-01,  1.9195e+00, -8.8630e-01],
        [-7.8070e-01,  1.9359e+00,  8.6450e-01],
        [ 6.4760e-01, -3.0820e-01, -6.9040e-01],
        [ 4.0020e-01, -2.8650e-01,  1.0585e+00],
        [-2.3542e+00, -2.0480e+00,  2.1284e+00],
        [-3.7422e+00, -1.8716e+00,  1.0105e+00],
        [-1.0629e+00, -2.9456e+00,  1.2640e-01],
        [-3.1162e+00, -2.4671e+00, -1.7246e+00],
        [-2.9009e+00, -9.1000e-03, -3.0579e+00],
        [ 1.2284e+00,  1.3310e-01,  1.2598e+00],
        [ 2.7420e-01,  3.5750e-01,  8.0700e-02],
        [ 7.9380e-01, -2.3460e-01, -1.2351e+00],
        [ 8.3220e-01, -1.7824e+00, -1.3006e+00],
        [ 1.1412e+00, -2.0311e+00, -2.8042e+00],
        [-2.8070e-01, -1.4303e+00, -2.9865e+00],
        [-6.3720e-01, -2.1531e+00, -1.6559e+00],
        [-6.6500e-02,  3.5100e-02, -2.5392e+00],
        [-1.3557e+00,  8.4370e-01, -2.3843e+00],
        [ 1.3701e+00, -9.3200e-01,  1.4691e+00],
        [ 2.2159e+00,  5.6260e-01,  1.0553e+00],
        [ 8.4550e-01,  6.0040e-01,  2.1726e+00],
        [-7.0970e-01, -5.9700e-02,  3.2660e-01],
        [ 1.1930e-01,  1.4357e+00, -4.7500e-02],
        [ 1.8083e+00,  1.5600e-01, -1.3879e+00],
        [ 1.3804e+00, -2.3016e+00, -5.1370e-01],
        [ 1.9866e+00, -1.4980e+00, -3.2515e+00],
        [ 1.1890e+00, -3.0934e+00, -3.0563e+00],
        [-8.6030e-01, -1.5839e+00, -3.8986e+00],
        [-7.7180e-01, -3.2291e+00, -1.7908e+00],
        [-1.4480e+00, -1.7578e+00, -1.0386e+00],
        [ 5.5770e-01,  5.4910e-01, -3.2800e+00],
        [-2.0611e+00,  3.8200e-01, -1.6870e+00],
        [-1.1566e+00,  1.8595e+00, -2.0283e+00],
        [-1.8619e+00,  9.3140e-01, -3.3521e+00],
        [-2.0740e-01,  1.5235e+00,  3.4610e-01],
        [ 5.1380e-01,  1.9230e-01,  3.5090e-01],
        [ 3.4350e-01, -7.3550e-01, -8.7480e-01],
        [ 1.8651e+00, -5.7450e-01, -1.0324e+00],
        [ 1.9396e+00,  3.3850e-01,  8.3800e-02],
        [ 2.7086e+00, -1.8365e+00, -8.5180e-01],
        [ 3.9895e+00, -1.6787e+00, -6.3600e-02],
        [ 2.3517e+00, -2.8817e+00, -1.3453e+00],
        [-7.5300e-02,  2.0343e+00, -6.1290e-01],
        [-1.2792e+00,  1.3759e+00,  5.1850e-01],
        [ 1.7990e-01,  2.1724e+00,  1.1377e+00],
        [ 3.6380e-01, -3.1770e-01,  1.3121e+00],
        [ 1.4400e-02, -1.7560e+00, -6.7970e-01],
        [-2.5080e-01, -3.0920e-01, -1.6845e+00],
        [ 2.1973e+00, -9.3300e-02, -1.9652e+00],
        [ 3.7603e+00, -1.3145e+00,  9.4200e-01],
        [ 4.5201e+00, -2.6303e+00, -2.0400e-02],
        [ 4.6246e+00, -9.1470e-01, -5.2710e-01],
        [-6.8000e-02,  1.6891e+00,  2.2520e-01],
        [-5.8400e-02,  1.8240e-01,  7.2600e-02],
        [-1.3779e+00, -4.2700e-01, -4.0830e-01],
        [-1.7462e+00,  4.9000e-03, -1.7532e+00],
        [-2.0338e+00,  3.6930e-01, -2.8616e+00],
        [ 1.2093e+00, -5.0060e-01, -3.4100e-01],
        [ 7.0810e-01, -6.6360e-01,  1.0908e+00],
        [ 1.4157e+00, -4.3500e-02,  2.1743e+00],
        [ 1.9811e+00,  4.5980e-01,  3.0498e+00],
        [ 8.9230e-01,  2.0619e+00,  5.8890e-01],
        [-2.8140e-01,  2.1610e+00, -7.3830e-01],
        [-8.3940e-01,  2.0049e+00,  9.3610e-01],
        [-1.3081e+00, -1.5218e+00, -3.8740e-01],
        [-2.1771e+00, -1.5710e-01,  2.9470e-01],
        [-2.2972e+00,  6.8530e-01, -3.8408e+00],
        [ 1.1528e+00, -1.3776e+00, -9.7630e-01],
        [ 2.0941e+00,  1.0810e-01, -4.8620e-01],
        [ 2.7610e-01, -1.6254e+00,  1.3484e+00],
        [-5.0100e-02,  1.1259e+00,  6.3970e-01],
        [-3.1590e-01, -3.8120e-01,  7.4300e-01],
        [-1.5226e+00, -7.8100e-01, -7.0700e-02],
        [-1.3585e+00, -8.5910e-01, -1.4923e+00],
        [-1.6999e+00, -2.0582e+00, -7.8760e-01],
        [-7.0910e-01, -3.1828e+00, -8.5060e-01],
        [ 3.1250e-01, -3.3016e+00,  1.4390e-01],
        [-7.1220e-01, -4.3013e+00,  1.0770e-01],
        [-1.6383e+00, -4.3811e+00,  1.2943e+00],
        [ 8.2340e-01,  1.4103e+00,  1.2335e+00],
        [ 1.3700e-01,  1.4141e+00, -3.9930e-01],
        [-9.0420e-01,  1.7101e+00,  1.0005e+00],
        [ 5.5400e-01, -9.4970e-01,  4.1130e-01],
        [-4.9370e-01, -6.6090e-01,  1.7893e+00],
        [-2.4462e+00, -2.7230e-01,  2.2240e-01],
        [-2.7268e+00, -2.3850e+00, -9.7370e-01],
        [-3.5960e-01, -3.3853e+00, -1.8650e+00],
        [-3.6060e-01, -5.2656e+00, -2.6410e-01],
        [-1.9862e+00, -3.3905e+00,  1.5955e+00],
        [-2.5097e+00, -5.0009e+00,  1.0568e+00],
        [-1.1251e+00, -4.8389e+00,  2.1461e+00],
        [ 2.0290e-01, -1.0722e+00, -2.0510e-01],
        [ 6.5300e-02,  1.1450e-01, -8.3000e-02],
        [-8.5000e-02,  1.5195e+00,  6.5900e-02],
        [-1.1518e+00,  2.1842e+00, -1.2110e-01],
        [-8.3570e-01,  3.5833e+00,  1.5470e-01],
        [ 6.8910e-01,  3.5786e+00,  5.1680e-01],
        [ 1.0516e+00,  2.1854e+00,  4.5960e-01],
        [ 1.5192e+00,  4.3462e+00, -4.1610e-01],
        [ 2.1478e+00,  4.9819e+00, -1.1475e+00],
        [ 3.1710e-01, -2.1230e+00, -3.1710e-01],
        [-1.0501e+00,  4.2034e+00, -7.1960e-01],
        [-1.4441e+00,  3.9519e+00,  9.8590e-01],
        [ 8.7990e-01,  3.9357e+00,  1.5325e+00],
        [ 1.2870e-01,  1.4367e+00, -2.8000e-02],
        [-4.5200e-02, -3.5000e-02,  3.1700e-02],
        [-1.1021e+00, -8.7080e-01,  2.2480e-01],
        [-6.6420e-01, -2.1813e+00,  1.7950e-01],
        [ 6.1430e-01, -2.0877e+00, -3.4900e-02],
        [ 1.0616e+00, -8.1630e-01, -1.3590e-01],
        [ 1.6014e+00, -3.1962e+00, -1.7570e-01],
        [ 1.0185e+00, -4.4177e+00, -5.3400e-02],
        [ 2.7833e+00, -3.0005e+00, -3.7600e-01],
        [ 8.2070e-01,  1.7869e+00,  7.4540e-01],
        [ 5.3120e-01,  1.7505e+00, -9.9700e-01],
        [-8.3450e-01,  1.9290e+00,  1.2180e-01],
        [-2.1372e+00, -6.1770e-01,  3.9150e-01],
        [ 2.9000e-02, -4.4958e+00,  1.1310e-01],
        [ 1.5941e+00, -5.2382e+00, -1.3130e-01],
        [ 8.6000e-03,  1.5795e+00,  3.2410e-01],
        [-1.2100e-02,  8.6100e-02, -5.1600e-02],
        [-1.3917e+00, -5.0560e-01, -7.8600e-02],
        [-2.0746e+00, -9.6280e-01, -1.1377e+00],
        [-1.6700e+00, -1.0179e+00, -2.5773e+00],
        [-3.4619e+00, -1.4248e+00, -7.4180e-01],
        [-3.7632e+00, -2.6623e+00, -1.3845e+00],
        [-3.3870e+00, -1.5205e+00,  7.9730e-01],
        [-2.2124e+00, -6.0210e-01,  1.1948e+00],
        [ 1.0339e+00,  1.9621e+00,  3.3960e-01],
        [-5.6390e-01,  2.1720e+00, -3.9670e-01],
        [-4.2470e-01,  1.7496e+00,  1.3147e+00],
        [ 4.7770e-01, -4.7800e-02, -1.0212e+00],
        [ 5.9590e-01, -4.6580e-01,  6.7970e-01],
        [-1.7858e+00, -2.0347e+00, -2.9662e+00],
        [-2.3182e+00, -3.7670e-01, -3.1901e+00],
        [-6.3760e-01, -6.9810e-01, -2.7365e+00],
        [-4.2047e+00, -6.6940e-01, -1.0570e+00],
        [-4.6932e+00, -2.8555e+00, -1.2330e+00],
        [-4.3271e+00, -1.2651e+00,  1.2954e+00],
        [-3.1484e+00, -2.5613e+00,  1.0389e+00],
        [-1.6311e+00, -9.9360e-01,  2.0392e+00],
        [-2.5689e+00,  3.8960e-01,  1.5068e+00]])
batch = torch.Tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,
         5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,
         6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
         7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11,
        11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
        14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
        15, 15, 15, 15, 15, 15, 15, 15, 15, 15])
e = torch.Tensor([3.7399, 2.7743, 5.0626, 3.6829, 2.0692, 4.0459, 2.9926, 2.8188, 4.3407,
        6.2906, 4.1168, 3.9576, 4.8980, 2.4191, 3.0708, 5.5749])

In [11]:
# In the hidden cell I've prepared 3 tensors that Physnet expects for input on a forward pass.
# They were pulled from the QM9 dataset, as the source notebook shows here: https://colab.research.google.com/drive/1viGlpUIGmRW-kzqERj4fGzX9udEXVrXw?usp=sharing#scrollTo=OXLcy3ZJ6pgH
print(z.shape, pos.shape, batch.shape)
# And this vector of constants we'll use for post-processing
print(e.shape)

torch.Size([280]) torch.Size([280, 3]) torch.Size([280])
torch.Size([16])


In [7]:
molecular_property_garden = garden_client.get_published_garden("10.23677/cneg-mq41")

In [12]:
result = molecular_property_garden.zpve_predict(z, pos, batch)

Output()

In [13]:
final_values = result[0].view(-1) - e

In [14]:
final_values

tensor([-0.0032, -0.0167,  0.0074,  0.0112,  0.0075,  0.0048,  0.0074, -0.0202,
         0.0098,  0.0123,  0.0145,  0.0069,  0.0220, -0.0047, -0.0038,  0.0065],
       grad_fn=<SubBackward0>)

These are close/identical to the values I got from running this forward pass in the context of the original notebook:

`[-0.0031, -0.0167,  0.0074,  0.0112,  0.0075,  0.0048,  0.0074, -0.0203, 0.0098,  0.0123,  0.0145,  0.0069,  0.0220, -0.0047, -0.0038,  0.0065]`